In [2]:
!pip install pandas
!pip install selenium
!pip install webdriver_manager
!pip install lxml
!pip install yfinance
!pip install tensorflow
!pip install scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, r2_score

In [4]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [5]:
# Scrap the most exchange value from SET
def scrape_industry_data(driver):
    
    industry_urls = {
        "agro": "https://www.set.or.th/th/market/index/set/agro",
        "consump": "https://www.set.or.th/th/market/index/set/consump",
        "fincial": "https://www.set.or.th/th/market/index/set/fincial",
        "indus": "https://www.set.or.th/th/market/index/set/indus",
        "propcon": "https://www.set.or.th/th/market/index/set/propcon",
        "resource": "https://www.set.or.th/th/market/index/set/resourc",
        "service": "https://www.set.or.th/th/market/index/set/service",
        "tech": "https://www.set.or.th/th/market/index/set/tech"
    }

    industry_data = {}

    for industry, url in industry_urls.items():
        print(f"Scraping {industry} data from {url}...")
        driver.get(url)
        sleep(5)  
        driver.execute_script("window.scrollTo(0, 1500)")  
        sleep(5)  
    
        retry_attempts = 3  
        data_cleaned = None  

        for attempt in range(retry_attempts):
            try:
                data_list = pd.read_html(driver.page_source)
                if len(data_list) > 1:
                    data_cleaned = data_list[1].dropna(how="all")  # Clean the data

            # If data_cleaned is not empty, break the retry loop
                if data_cleaned is not None and not data_cleaned.empty:
                    print(data_cleaned)
                    industry_data[industry] = data_cleaned  # Store in dictionary
                    print(f"Successfully scraped {industry} data.")
                    break
                else:
                    print(f"No data found for {industry}. Retrying... (Attempt {attempt + 1})")
                    sleep(5)  # Wait before retrying
                    driver.refresh()  # Refresh the page

            except Exception as e:
                print(f"Error scraping {industry} on attempt {attempt + 1}: {e}")
        else:
            print(f"Failed to scrape data for {industry} after {retry_attempts} attempts.")

    return industry_data

# Get the top 5
def get_top_5_by_value(dataframe):
    if dataframe.empty or "มูลค่า ('000 บาท)" not in dataframe.columns:
        return pd.DataFrame(columns=["หลักทรัพย์", "มูลค่า ('000 บาท)"])  # Return empty DataFrame if missing data

    dataframe["มูลค่า ('000 บาท)"] = pd.to_numeric(dataframe["มูลค่า ('000 บาท)"], errors="coerce")
    
    dataframe = dataframe.dropna(subset=["มูลค่า ('000 บาท)"])

    top_5 = dataframe.sort_values(by="มูลค่า ('000 บาท)", ascending=False).head(5)

    return top_5[["หลักทรัพย์", "มูลค่า ('000 บาท)"]]

# Extract for furthur input
def extract_top_5_data_with_tickers(source_data):
    top_5_industries = {industry: get_top_5_by_value(df) for industry, df in source_data.items()}
    
    # Extract only the top 5 names and add ".BK"
    top_5_names_industries = {
        industry: [f"{ticker}.BK" for ticker in df["หลักทรัพย์"].tolist()]
        for industry, df in top_5_industries.items()
    }

    return top_5_industries, top_5_names_industries

In [6]:
# Call function to scrape the data from SET
industry_data = scrape_industry_data(driver)

Scraping agro data from https://www.set.or.th/th/market/index/set/agro...


/var/folders/vw/288jn05573d78l0mr6f151zc0000gn/T/ipykernel_87247/2857208460.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_list = pd.read_html(driver.page_source)


                         หลักทรัพย์   เปิด สูงสุด ต่ำสุด ล่าสุด เปลี่ยนแปลง  \
1    AGRO - เกษตรและอุตสาหกรรมอาหาร    NaN    NaN    NaN    NaN         NaN   
2             AGRI - ธุรกิจการเกษตร    NaN    NaN    NaN    NaN         NaN   
3                        EE  CB  CS   0.49   0.52   0.48   0.50        0.00   
5                              GFPT   8.60   8.80   8.55   8.70       +0.10   
7                               LEE   2.28   2.28   2.22   2.28        0.00   
..                              ...    ...    ...    ...    ...         ...   
134                              TU  11.50  11.60  11.40  11.50        0.00   
136                             TVO  20.70  21.00  20.70  21.00       +0.30   
138                            TWPC   2.22   2.22   2.18   2.20       -0.02   
140                        XBIO  CB   0.29   0.30   0.29   0.29       -0.01   
142                             ZEN   6.60   7.00   6.60   6.95       +0.35   

    % เปลี่ยนแปลง  เสนอซื้อ  เสนอขาย ปริมาณ (หุ้น) 

/var/folders/vw/288jn05573d78l0mr6f151zc0000gn/T/ipykernel_87247/2857208460.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_list = pd.read_html(driver.page_source)


                             หลักทรัพย์    เปิด  สูงสุด  ต่ำสุด  ล่าสุด  \
1          CONSUMP - สินค้าอุปโภคบริโภค     NaN     NaN     NaN     NaN   
2                      FASHION - แฟชั่น     NaN     NaN     NaN     NaN   
3                                   AFC    3.70    3.90    3.58    3.74   
5                                  AURA   14.00   14.10   13.80   14.00   
7                                  BTNC       -       -       -       -   
9                                   CPH    7.80    7.90    7.80    7.90   
11                                  CPL    0.90    0.90    0.84    0.89   
13                                   NC    2.52    2.52    2.36    2.38   
15                                  PAF    0.90    0.90    0.84    0.87   
17                                  PDJ    1.84    1.84    1.64    1.82   
19                                   PG    9.00    9.05    9.00    9.05   
21                               SABINA   18.20   18.40   18.10   18.30   
23                       

...

           หลักทรัพย์  เปิด สูงสุด ต่ำสุด ล่าสุด เปลี่ยนแปลง % เปลี่ยนแปลง  \
1    SERVICE - บริการ   NaN    NaN    NaN    NaN         NaN           NaN   
2      COMM - พาณิชย์   NaN    NaN    NaN    NaN         NaN           NaN   
3              ADVICE  4.66   4.94   4.66   4.90       +0.20         +4.26   
5             B52  CB  0.28   0.30   0.27   0.29       +0.01         +3.57   
7              BEAUTY  0.22   0.23   0.21   0.23       +0.01         +4.55   
..                ...   ...    ...    ...    ...         ...           ...   
268             TFFIF  6.30   6.40   6.30   6.30        0.00          0.00   
270  THAI  SP  NC  NP     -      -      -      -           -             -   
272              TSTE     -      -      -      -           -             -   
274               TTA  4.04   4.30   4.04   4.28       +0.24         +5.94   
276              WICE  3.30   3.30   3.24   3.30        0.00          0.00   

    เสนอซื้อ เสนอขาย ปริมาณ (หุ้น) มูลค่า ('000 บาท)  
1       

/var/folders/vw/288jn05573d78l0mr6f151zc0000gn/T/ipykernel_87247/2857208460.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_list = pd.read_html(driver.page_source)


No data found for tech. Retrying... (Attempt 1)


/var/folders/vw/288jn05573d78l0mr6f151zc0000gn/T/ipykernel_87247/2857208460.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_list = pd.read_html(driver.page_source)


                              หลักทรัพย์    เปิด  สูงสุด  ต่ำสุด  ล่าสุด  \
1                       TECH - เทคโนโลยี     NaN     NaN     NaN     NaN   
2         ETRON - ชิ้นส่วนอิเล็กทรอนิกส์     NaN     NaN     NaN     NaN   
3                                   CCET    7.45    8.20    7.40    8.10   
5                                  DELTA  115.00  115.00  112.50  114.00   
7                                   HANA   20.90   21.90   20.90   21.80   
9                                    KCE   19.80   21.20   19.70   21.20   
11                                 METCO  161.00  161.00  158.50  159.00   
13                                   SMT    1.19    1.21    1.18    1.20   
15                                   SVI    7.05    7.15    7.00    7.00   
17                                  TEAM    2.58    2.62    2.56    2.56   
19  ICT - เทคโนโลยีสารสนเทศและการสื่อสาร     NaN     NaN     NaN     NaN   
20                                 3BBIF    5.40    5.45    5.35    5.45   
22          

In [10]:
# Print results
def get_top_5_tickers():
    top_5_industries = {
        industry: get_top_5_by_value(df) for industry, df in industry_data.items()
    }
    top_5_industries, top_5_names_industries = extract_top_5_data_with_tickers(industry_data)
    return top_5_names_industries  

top_5_names_industries = get_top_5_tickers()

print("\n========= TOP 5 for Pulling from Yahoo Finance =========")
print(top_5_names_industries)


========= TOP 5 for Pulling from Yahoo Finance =========
{'agro': ['CBG.BK', 'OSP.BK', 'OKJ.BK', 'CPF.BK', 'TU.BK'], 'consump': ['SNPS.BK', 'AURA.BK', 'SABINA.BK', 'STGT.BK', 'TMAN.BK'], 'fincial': ['KBANK.BK', 'KTB.BK', 'SCB.BK', 'BBL.BK', 'TTB.BK'], 'indus': ['IVL.BK', 'PTTGC.BK', 'SCGP.BK', 'RSXYZ.BK', 'TPCS.BK'], 'propcon': ['AMATA.BK', 'CPN.BK', 'WHA.BK', 'SIRI.BK', 'SCC.BK'], 'resource': ['GULF.BK', 'PTTEP.BK', 'TOP.BK', 'PTT.BK', 'BCP.BK'], 'service': ['CPALL.BK', 'AOT.BK', 'BH.BK', 'BDMS.BK', 'MINT.BK'], 'tech': ['ADVANC.BK', 'INTUCH.BK', 'DELTA.BK', 'CCET.BK', 'TRUE.BK']}


In [11]:
import yfinance as yf

# Download historical data for all stocks
end_date = datetime.datetime.today()
start_date = end_date - datetime.timedelta(days=730)

def get_40_stocks_data(top_5_names_industries):
    all_data = {}
    for industry, tickers in top_5_names_industries.items():
        print(f"Downloading data for {industry} stocks...")
        for ticker in tickers:
            data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'), auto_adjust=True)
            all_data[ticker] = data  # Store data by ticker
            print(f"Downloaded data for {ticker} with {len(data)} rows")

    return all_data

all_data = get_40_stocks_data(top_5_names_industries)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded data for CBG.BK
Downloaded data for OSP.BK
Downloaded data for OKJ.BK


[*********************100%***********************]  1 of 1 completed


Downloaded data for CPF.BK


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

Downloaded data for ADVANC.BK
Downloaded data for INTUCH.BK


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded data for DELTA.BK
Downloaded data for CCET.BK


[*********************100%***********************]  1 of 1 completed

Downloaded data for TRUE.BK


In [ ]:
# Get close price function from each of the stock
def get_close_price(stock):
    return all_data[stock][['Close']]

In [12]:
all_data

{'CBG.BK': Price           Close       High        Low       Open   Volume
 Ticker         CBG.BK     CBG.BK     CBG.BK     CBG.BK   CBG.BK
 Date                                                           
 2024-02-13  71.358871  72.343132  69.144286  69.636416  5200900
 2024-02-14  71.358871  71.358871  70.128546  70.374611  2908600
 2024-02-15  72.343140  72.835270  70.374619  71.604944  5134600
 2024-02-16  72.097069  73.081329  71.604939  72.835264  3318700
 2024-02-19  71.358871  71.604937  70.128546  70.866741  8088000
 ...               ...        ...        ...        ...      ...
 2025-02-05  73.500000  74.250000  72.500000  73.750000  3764500
 2025-02-06  71.500000  74.000000  70.250000  74.000000  4220300
 2025-02-07  71.250000  72.500000  70.000000  72.000000  4117400
 2025-02-10  68.250000  71.750000  68.000000  71.000000  3504600
 2025-02-11  71.500000  71.500000  68.250000  68.250000  4619400
 
 [243 rows x 5 columns],
 'OSP.BK': Price           Close       High        Lo

In [ ]:
# Example of one data history
data_true = get_close_price("TRUE.BK")

data_true['Close'].plot()
plt.title("TRUE Stock Prices")
plt.show()

In [15]:
# Save data to SCV file
import os

def save_stock_data(all_data, top_5_names_industries):
    base_path = "data"

    for industry, tickers in top_5_names_industries.items():
        industry_path = os.path.join(base_path, industry)
        os.makedirs(industry_path, exist_ok=True)  # Create industry folder if not exists
        
        for ticker in tickers:
            df = all_data.get(ticker)
            if df is not None and not df.empty:
                file_path = os.path.join(industry_path, f"{ticker}.csv")
                df.to_csv(file_path)
                print(f"Saved {ticker} data to {file_path}")

save_stock_data(all_data, top_5_names_industries)

Saved CBG.BK data to data/agro/CBG.BK.csv
Saved OSP.BK data to data/agro/OSP.BK.csv
Saved OKJ.BK data to data/agro/OKJ.BK.csv
Saved CPF.BK data to data/agro/CPF.BK.csv
Saved TU.BK data to data/agro/TU.BK.csv
Saved SNPS.BK data to data/consump/SNPS.BK.csv
Saved AURA.BK data to data/consump/AURA.BK.csv
Saved SABINA.BK data to data/consump/SABINA.BK.csv
Saved STGT.BK data to data/consump/STGT.BK.csv
Saved TMAN.BK data to data/consump/TMAN.BK.csv
Saved KBANK.BK data to data/fincial/KBANK.BK.csv
Saved KTB.BK data to data/fincial/KTB.BK.csv
Saved SCB.BK data to data/fincial/SCB.BK.csv
Saved BBL.BK data to data/fincial/BBL.BK.csv
Saved TTB.BK data to data/fincial/TTB.BK.csv
Saved IVL.BK data to data/indus/IVL.BK.csv
Saved PTTGC.BK data to data/indus/PTTGC.BK.csv
Saved SCGP.BK data to data/indus/SCGP.BK.csv
Saved RSXYZ.BK data to data/indus/RSXYZ.BK.csv
Saved TPCS.BK data to data/indus/TPCS.BK.csv
Saved AMATA.BK data to data/propcon/AMATA.BK.csv
Saved CPN.BK data to data/propcon/CPN.BK.csv
Sav